In [34]:
with open("seed.txt", "r") as f:
    seed = int(f.read().strip())

print(f"seed = {seed}")

seed = 52772


In [35]:
#Task 1: Regression — Regularization on Housing Data
#1.1 1A. Data Preprocessing (Required)
#1. Use your seed to create a 60/20/20 train/validation/test split.

#load dataset
import urllib.request
import pandas as pd
import numpy as np

url = "https://lib.stat.cmu.edu/datasets/boston"
response = urllib.request.urlopen(url)
raw = response.read().decode()
lines = raw.strip().splitlines()
data_lines = lines[22:] #dataset starts from line 23 

dataset = []

#each data contains 2 lines
for i in range(0, len(data_lines), 2):
    line1 = data_lines[i].strip().split()
    line2 = data_lines[i+1].strip().split()
    dataset.append(line1 + line2)

column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM",
    "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

df = pd.DataFrame(dataset, columns=column_names).astype(float)

#display(df)

total_rows = len(df)
print(f"Total rows in dataset: {total_rows}")

np.random.seed(seed)
indices = np.random.permutation(total_rows)

train_end = int(0.6 * total_rows)
validation_end = int(0.8 * total_rows)

train_index = indices[:train_end] #60%
validation_index   = indices[train_end:validation_end] #20%
test_index  = indices[validation_end:] #20%

train_df = df.iloc[train_index]
validation_df   = df.iloc[validation_index]
test_df  = df.iloc[test_index]

#verify split correctly
print(f"Train set: {len(train_df)} rows,", f"Validation set: {len(validation_df)} rows,", f"Test set: {len(test_df)}")
print(f"Train: {len(train_df)/len(df)*100:.1f}%, ", f"Validation: {len(validation_df)/len(df)*100:.1f}%, ", f"Test: {len(test_df)/len(df)*100:.1f}%")


Total rows in dataset: 506
Train set: 303 rows, Validation set: 101 rows, Test set: 102
Train: 59.9%,  Validation: 20.0%,  Test: 20.2%


In [48]:
#2. Handle missing values (if any) and apply appropriate transformations:
#• If categorical columns exist: apply one-hot encoding.

df.isnull().sum()
#So there is no missing values in this dataset

categorical_columns = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
print(categorical_columns)
#categorical_columns is empty, so there is no categorical columns in this dataset


#• Create two interaction features of your choice; justify why they may help.

# RM(average number of rooms per dwelling)and LSTAT(% lower status of the population) are two interaction features. 
# In a high-status neighborhood, where LSTAT is low, an extra room can add a lot of value. However, in a low-status area,
# where LSTAT is high, an extra room may only add little value. So in the model, we want to show that the effect of room 
# numbers depends on the LSTAT. Adding the interaction RM × LSTAT to capture non-additive effects between house size and 
# neighborhood socioeconomic status can improve model expressiveness. 



[]


In [56]:
#3. Run the full pipeline under three scaling settings:

X_train = train_df.to_numpy(dtype=float)
X_validation = validation_df.to_numpy(dtype=float)
X_test = test_df.to_numpy(dtype=float)

#(a) No scaling
train_no_scale, validation_no_scale, test_no_scale = X_train, X_validation, X_test

#(b) Standard scaling (z-score)
def standard_scaler_fit(X_train):
    mu = X_train.mean(axis=0)
    sigma = X_train.std(axis=0, ddof=0)
    sigma[sigma == 0] = 1.0  # signma cannot be zero
    return mu, sigma

def standard_scaler_transform(X, mu, sigma):
    return (X - mu) / sigma

mu, sigma = standard_scaler_fit(X_train)

train_standard_scaling = standard_scaler_transform(X_train, mu, sigma)
validation_standard_scaling = standard_scaler_transform(X_validation, mu, sigma)
test_standard_scaling = standard_scaler_transform(X_test, mu, sigma)

#(c) Robust scaling (median/IQR)
def robust_scaler_fit(X_train):
    median = np.median(X_train, axis=0)
    q1 = np.percentile(X_train, 25, axis=0)
    q3 = np.percentile(X_train, 75, axis =0)
    iqr =  q3 - q1
    iqr[iqr == 0] = 1 # iqr cannot be zero
    return median, iqr

def robust_scaler_transform(X, median, iqr):
    return (X-median) / iqr

median, iqr = standard_scaler_fit(X_train)

train_robust_scaling = robust_scaler_transform(X_train, median, iqr)
validation_robust_scaling = robust_scaler_transform(X_validation, median, iqr)
test_robust_scaling = robust_scaler_transform(X_test, median, iqr)

print("No Scaling: ", train_no_scale.shape, validation_no_scale.shape, test_no_scale.shape)
print("Standard Scaling: ", train_no_scale.shape, validation_no_scale.shape, test_no_scale.shape)
print("Robust Scaling: ", train_no_scale.shape, validation_no_scale.shape, test_no_scale.shape)


No Scale:  (303, 14) (101, 14) (102, 14)
Standard Scale:  (303, 14) (101, 14) (102, 14)
No scale:  (303, 14) (101, 14) (102, 14)
